In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Reading the Dataset
dataset = pd.read_csv('Social_Network_Ads.csv')

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [5]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [6]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [7]:
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dep=dataset['Purchased']

In [8]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
from sklearn.svm import SVC
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [1.0, 10.0, 100.0],  
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': [3],            
    'gamma': ['scale', 'auto'],
    'coef0': [0.0],           
    'shrinking': [True],      
    'probability': [True],    
    'tol': [0.001],           
    'cache_size': [200.0],    
    'class_weight': [None, 'balanced'], 
    'verbose': [False],       
    'max_iter': [-1],         
    'decision_function_shape': ['ovo', 'ovr'],
    'break_ties': [False],    
    'random_state': [None]    
}

# Initialize SVC
svc_model = SVC()

# Initialize GridSearchCV with the corrected param_grid
# Add error_score='raise' for detailed error messages if issues persist
grid = GridSearchCV(svc_model, param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted', error_score='raise')

# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


GridSearchCV(error_score='raise', estimator=SVC(), n_jobs=-1,
             param_grid={'C': [1.0, 10.0, 100.0], 'break_ties': [False],
                         'cache_size': [200.0],
                         'class_weight': [None, 'balanced'], 'coef0': [0.0],
                         'decision_function_shape': ['ovo', 'ovr'],
                         'degree': [3], 'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                         'max_iter': [-1], 'probability': [True],
                         'random_state': [None], 'shrinking': [True],
                         'tol': [0.001], 'verbose': [False]},
             scoring='f1_weighted', verbose=3)

In [11]:
# print best parameter after tuning 
#print(grid.best_params_) 
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test) 
   
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [12]:

from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'C': 100.0, 'break_ties': False, 'cache_size': 200.0, 'class_weight': 'balanced', 'coef0': 0.0, 'decision_function_shape': 'ovo', 'degree': 3, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}: 0.9180821729616219


In [13]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[79  6]
 [ 5 44]]


In [14]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.94      0.93      0.93        85
           1       0.88      0.90      0.89        49

    accuracy                           0.92       134
   macro avg       0.91      0.91      0.91       134
weighted avg       0.92      0.92      0.92       134



In [15]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9553421368547419

In [16]:
table=pd.DataFrame.from_dict(re)

In [17]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_break_ties,param_cache_size,param_class_weight,param_coef0,param_decision_function_shape,...,param_verbose,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.439756,0.212662,0.109495,0.048763,1.0,False,200.0,None,0.0,ovo,...,False,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.776290,0.770070,0.698235,0.923510,0.901744,0.813970,0.085375,61
1,0.014982,0.001634,0.004789,0.000746,1.0,False,200.0,None,0.0,ovo,...,False,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.799620,0.804628,0.705612,0.943041,0.813763,0.813333,0.075752,65
2,0.013766,0.000747,0.005188,0.000746,1.0,False,200.0,None,0.0,ovo,...,False,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.887346,0.886792,0.850543,0.944161,0.962264,0.906221,0.041022,13
3,0.014841,0.002250,0.004867,0.000783,1.0,False,200.0,None,0.0,ovo,...,False,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.732497,0.752940,0.626628,0.905069,0.826499,0.768727,0.093467,75
4,0.010973,0.000630,0.004589,0.000489,1.0,False,200.0,None,0.0,ovo,...,False,"{'C': 1.0, 'break_ties': False, 'cache_size': ...",0.776290,0.770070,0.698235,0.923510,0.901744,0.813970,0.085375,61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,0.010574,0.000798,0.004401,0.000820,100.0,False,200.0,balanced,0.0,ovr,...,False,"{'C': 100.0, 'break_ties': False, 'cache_size'...",0.723771,0.649581,0.703055,0.778067,0.720804,0.715055,0.041256,91
92,0.046831,0.017534,0.002784,0.000755,100.0,False,200.0,balanced,0.0,ovr,...,False,"{'C': 100.0, 'break_ties': False, 'cache_size'...",0.777778,0.834012,0.795256,0.815056,0.851527,0.814726,0.026346,57
93,0.064252,0.014153,0.003781,0.000394,100.0,False,200.0,balanced,0.0,ovr,...,False,"{'C': 100.0, 'break_ties': False, 'cache_size'...",0.826263,0.765553,0.832483,0.869709,0.923510,0.843503,0.052120,37
94,0.018512,0.001670,0.004714,0.000710,100.0,False,200.0,balanced,0.0,ovr,...,False,"{'C': 100.0, 'break_ties': False, 'cache_size'...",0.906808,0.944306,0.870362,0.925272,0.943041,0.917958,0.027447,1


In [18]:
Age=float(input("Age:"))
EstimatedSalary=float(input("Salary:"))
Gender=float(input("Gender:"))

Age: 38
Salary: 78000
Gender: 1


In [19]:
Future_Prediction=grid.predict([[Age,EstimatedSalary,Gender]])# change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))


Future_Prediction=[1]
